In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm
from molsysmt import puw

# Adding solvent

In [3]:
molsys = msm.convert('pdbid:181L', to_form='molsysmt.MolSys')
molsys = msm.extract(molsys, selection='molecule_type=="protein"')
molsys = msm.add_terminal_capping(molsys, N_terminal='ACE', C_terminal='NME')
molsys = msm.add_missing_hydrogens(molsys)

/home/diego/Projects/MolSysMT/molsysmt/native/io/topology/classes/mmtf_MMTFDecoder.py:43: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33
  warnings.warn(warning_message)


In [4]:
msm.info(molsys)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_proteins,n_frames
molsysmt.MolSys,2612,164,1,1,1,1,1,1


In [5]:
msm.physchem.charge([molsys, {'forcefield':'AMBER14'}], target='system')

8.0 <Unit('elementary_charge')>

In [6]:
msm.is_solvated(molsys)

False

In [7]:
molsys_cub = msm.solvate([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}],
                          box_geometry='cubic', clearance=14.0*puw.unit('angstroms'),
                          to_form='molsysmt.MolSys', engine="OpenMM", verbose=False)

In [8]:
msm.is_solvated(molsys_cub)

True

In [9]:
msm.info(molsys_cub)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_ions,n_proteins,n_frames
molsysmt.MolSys,45184,14360,14197,3,14197,3,14188,8,1,1


In [10]:
msm.info(molsys_cub, target='entity')

index,name,type,n atoms,n groups,n components,n chains,n molecules
0,Protein_0,protein,2612,164,1,1,1
1,water,water,42564,14188,14188,1,14188
2,CL,ion,8,8,8,1,8


In [11]:
msm.physchem.charge([molsys_cub, {'forcefield':'AMBER14', 'water_model':'TIP3P'}], target='system')

-0.0 <Unit('elementary_charge')>

In [12]:
box, box_angles = msm.get(molsys_cub, target='system', box=True, box_angles=True)

In [13]:
box

Magnitude,[[[7.783335137367249 0.0 0.0] [0.0 7.783335137367249 0.0] [0.0 0.0 7.783335137367249]]]
Units,nanometer


In [14]:
box_angles

Magnitude,[[90.000001 90.000001 90.000001]]
Units,degree


In [15]:
molsys_cub = msm.wrap_to_pbc(molsys_cub, center_of_selection='molecule_type=="protein"')

In [16]:
msm.view(molsys_cub, standardize=True, water_as_surface=True)

NGLWidget()

## Adding ions 

## PBC box geometry

All periodic boxes used in molecular dynamics simulations (cubic, triclinic,  hexagonal, dodecahedral or octahedral) are equivalent equivalent. All of them can be transformed into a triclinic box with the proper angles and edge lengths. See: Bekker, H. “Unification of Box Shapes in Molecular Simulations.” Journal of Computational Chemistry 18, no. 15 (1997): 1930–42. https://doi.org/10.1002/(sici)1096-987x(19971130)18:15<1930::aid-jcc8>3.0.co;2-p.

In [17]:
molsys_oct = msm.solvate(molsys, box_geometry='truncated_octahedral',
                         clearance=14.0*puw.unit('angstroms'), engine='PDBFixer')

In [18]:
msm.info(molsys_oct)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_ions,n_proteins,n_frames
molsysmt.MolSys,19507,5801,5638,3,5638,3,5629,8,1,1


In [19]:
molsys_oct = msm.wrap_to_pbc(molsys_oct, center_of_selection='molecule_type=="protein"')

In [20]:
msm.view(molsys_oct, standardize=True, water_as_surface=True)

NGLWidget()

In a triclinic box it is not sure that all elements in the unit cell can be considered first neighbors. Some pairs of atoms minimize their distance when one of them are located in a neighbor unit cell. But ¿Which one? Finding the periodic image that minimizes the distance is not in general as straight forward as it is if the box is cubic. This problem is known as "the minimum image convention". Actually, the distance between any two atoms in a periodic box is not computed removing the PBC, or centering a unit cell in any of those atoms. It is solved finding the minimum image convention. Then, let's see what happens when we take only the image of every atom with minimal distance to the center of the protein:

In [30]:
molsys_oct = msm.wrap_to_mic(molsys_oct, center_of_selection='molecule_type=="protein"')

In [22]:
msm.view(molsys_oct, standardize=True, water_as_surface=True)

NGLWidget()

The equivalent geometry is now recovered. It is then "a proof" of the equivalency between the triclinic box and the truncated octahedral box.

But why do we need a non cubic periodic box? In general a case, we want to be sure that a molecule is "solvated". What does this mean? It means that our molecule is surrounded by a thick enough layer of water molecules. ¿This can be accomplished by a cubic periodic box? Yes of course. But it can also be achieved with other geometries making use of a lower number of water molecules. Which means that running a molecular simulation with these other geometries will be computationally cheaper than with a periodic cube:

In [23]:
molsys_cub = msm.solvate(molsys, box_geometry='cubic', clearance=20.0*puw.unit('angstroms'), engine='PDBFixer')
molsys_oct = msm.solvate(molsys, box_geometry='truncated_octahedral',  clearance=20.0*puw.unit('angstroms'), engine='PDBFixer')
molsys_dod = msm.solvate(molsys, box_geometry='rhombic_dodecahedral', clearance=20.0*puw.unit('angstroms'), engine='PDBFixer')

In [24]:
n_waters_cub = msm.get(molsys_cub, target='system', n_waters=True)
n_waters_oct = msm.get(molsys_oct, target='system', n_waters=True)
n_waters_dod = msm.get(molsys_dod, target='system', n_waters=True)

In [25]:
print('Cubic box: {} water ({}% -cubic reference-)'.format(n_waters_cub, 100.0* n_waters_cub/n_waters_cub))
print('Truncated octahedral box: {} water ({}% -cubic reference-)'.format(n_waters_oct, 100.0* n_waters_oct/n_waters_cub))
print('Rhombic dodecahedron box: {} water ({}% -cubic reference-)'.format(n_waters_dod, 100.0* n_waters_dod/n_waters_cub))

Cubic box: 8775 water (100.0% -cubic reference-)
Truncated octahedral box: 7686 water (87.58974358974359% -cubic reference-)
Rhombic dodecahedron box: 6867 water (78.25641025641026% -cubic reference-)


## Solvation engines

In [26]:
molsys_oct_leap = msm.solvate([msm.remove_hydrogens(molsys),
                              {'forcefield':'AMBER14', 'water_model':'TIP3P'}],
                              box_geometry='truncated_octahedral',
                              clearance=14.0*puw.unit('angstroms'),
                              to_form='molsysmt.MolSys', engine='LEaP', verbose=False)

In [27]:
msm.info(molsys_oct_leap)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_ions,n_proteins,n_frames
molsysmt.MolSys,36913,11603,11440,1,11440,3,11431,8,1,1


In [28]:
molsys_oct_leap = msm.wrap_to_mic(molsys_oct_leap, center_of_selection='molecule_type=="protein"')

In [29]:
msm.view(molsys_oct_leap, standardize=True, water_as_surface=True)

NGLWidget()